In [37]:
# here we use the fast implementation of kemeny-young using graphs and integer programming found here:
#http://vene.ro/blog/kemeny-young-optimal-rank-aggregation-in-python.html

from __future__ import print_function
import numpy as np
from itertools import combinations, permutations
from lp_solve import lp_solve
import csv

In [38]:
def kendalltau_dist(rank_a, rank_b):
    tau = 0
    n_candidates = len(rank_a)
    for i, j in combinations(range(n_candidates), 2):
        tau += (np.sign(rank_a[i] - rank_a[j]) ==
                -np.sign(rank_b[i] - rank_b[j]))
    return tau

In [39]:
#dummy data found at aforementioned website
cols = "Alicia Ginny Gwendolyn Robin Debbie".split()
ranks = np.array([[0, 1, 2, 3, 4],
                  [0, 1, 3, 2, 4],
                  [4, 1, 2, 0, 3],
                  [4, 1, 0, 2, 3],
                  [4, 1, 3, 2, 0]])

#sanity check: this should be zero
kendalltau_dist(ranks[0], ranks[0])

0

In [40]:
#matches what is expected
print('d(athletic_wizard, prophet) = {}'.format(kendalltau_dist(ranks[0], ranks[1])))
print('d(athletic_wizard, seeker) = {}'.format(kendalltau_dist(ranks[0], ranks[3])))

d(athletic_wizard, prophet) = 1
d(athletic_wizard, seeker) = 5


In [41]:
#graph-distance conversion of ranks
def _build_graph(ranks):
    n_voters, n_candidates = ranks.shape
    edge_weights = np.zeros((n_candidates, n_candidates))
    for i, j in combinations(range(n_candidates), 2):
        preference = ranks[:, i] - ranks[:, j]
        h_ij = np.sum(preference < 0)  # prefers i to j
        h_ji = np.sum(preference > 0)  # prefers j to i
        if h_ij > h_ji:
            edge_weights[i, j] = h_ij - h_ji
        elif h_ij < h_ji:
            edge_weights[j, i] = h_ji - h_ij
    return edge_weights

In [42]:
print(_build_graph(ranks))
#as expected from website. we are doing good!

[[ 0.  0.  0.  0.  0.]
 [ 1.  0.  3.  3.  3.]
 [ 1.  0.  0.  0.  3.]
 [ 1.  0.  1.  0.  3.]
 [ 1.  0.  0.  0.  0.]]


In [43]:
#function that actually calculates k-y ranks by implementing fancy integer programming
def rankaggr_lp(ranks):
    """Kemeny-Young optimal rank aggregation"""

    n_voters, n_candidates = ranks.shape
    
    # maximize c.T * x
    edge_weights = _build_graph(ranks)
    c = -1 * edge_weights.ravel()  

    idx = lambda i, j: n_candidates * i + j

    # constraints for every pair
    pairwise_constraints = np.zeros(((n_candidates * (n_candidates - 1)) / 2,
                                     n_candidates ** 2))
    for row, (i, j) in zip(pairwise_constraints,
                           combinations(range(n_candidates), 2)):
        row[[idx(i, j), idx(j, i)]] = 1

    # and for every cycle of length 3
    triangle_constraints = np.zeros(((n_candidates * (n_candidates - 1) *
                                     (n_candidates - 2)),
                                     n_candidates ** 2))
    for row, (i, j, k) in zip(triangle_constraints,
                              permutations(range(n_candidates), 3)):
        row[[idx(i, j), idx(j, k), idx(k, i)]] = 1

    constraints = np.vstack([pairwise_constraints, triangle_constraints])
    constraint_rhs = np.hstack([np.ones(len(pairwise_constraints)),
                                np.ones(len(triangle_constraints))])
    constraint_signs = np.hstack([np.zeros(len(pairwise_constraints)),  # ==
                                  np.ones(len(triangle_constraints))])  # >=

    obj, x, duals = lp_solve(c, constraints, constraint_rhs, constraint_signs,
                             xint=range(1, 1 + n_candidates ** 2))

    x = np.array(x).reshape((n_candidates, n_candidates))
    aggr_rank = x.sum(axis=1)

    return obj, aggr_rank

In [44]:
#here I play around with the data by screwing with the ranks to see if it's working as expected.
#I try same ranks for multiple people, same rank for all but one, etc to see how edge case behaviour pans out
cols = "Alicia Ginny Gwendolyn Robin Debbie".split()
ranks = np.array([[1, 1, 2, 3, 15],
                  [1, 1, 3, 2, 15],
                  [1, 1, 2, 0, 3],
                  [1, 1, 0, 2, 3],
                  [1, 1, 3, 2, 0]])

_, aggr = rankaggr_lp(ranks)
score = np.sum(kendalltau_dist(aggr, rank) for rank in ranks)
print("A Kemeny-Young aggregation with score {} is: {}".format(
    score,
    ", ".join(cols[i] for i in np.argsort(aggr))))

A Kemeny-Young aggregation with score 10 is: Ginny, Alicia, Robin, Gwendolyn, Debbie


In [45]:
#here I implement the wikipedia example to test if this implementation is robust--it is!
# https://en.wikipedia.org/wiki/Kemeny%E2%80%93Young_method#Example
city_names= ['M', 'N', 'C','K']

pref_1 = [1,2,3,4]
pref_2 = [4,1,2,3]
pref_3 = [4,3,1,2]
pref_4 = [4,3,2,1]

all_data = np.array([1,1,1,1])

for x in range(0,42):
    all_data = np.vstack((all_data, pref_1))

for x in range(0,26):
    all_data = np.vstack((all_data, pref_2))

for x in range(0,15):
    all_data = np.vstack((all_data, pref_3))

for x in range(0,17):
    all_data = np.vstack((all_data, pref_4))
    
    
_, aggr = rankaggr_lp(all_data)
score = np.sum(kendalltau_dist(aggr, rank) for rank in all_data)
print("A Kemeny-Young aggregation with score {} is: {}".format(
    score,
    ", ".join(city_names[i] for i in np.argsort(aggr))))

#notice here that the "score" is different from wikipedia. this is because we are counting kendall-tau distances now,
#not entries from a frequency table.

A Kemeny-Young aggregation with score 207 is: N, C, K, M


In [46]:
#and now to the YNC case!

names = ['Aadit', 'Adam', 'Annette', 'Diamanta', 'Isaac', 'Isobel', 'Jay', 'Joceline', 'Keith',\
         'Meredith', 'Regina', 'Ross', 'Scott', 'Thaddeus']

#we use the table we cleaned up in "clean_ky_method.ipynb"

with open('test_file.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    prefs = np.array([[int(e) for e in r] for r in reader])

#this gives the optimal ranking

_, aggr = rankaggr_lp(prefs)
score = np.sum(kendalltau_dist(aggr, rank) for rank in prefs)
print("A Kemeny-Young aggregation with score {} is: {}".format(
    score,
    ", ".join(names[i] for i in np.argsort(aggr))))

A Kemeny-Young aggregation with score 7785 is: Regina, Joceline, Annette, Meredith, Jay, Isobel, Diamanta, Isaac, Scott, Ross, Keith, Adam, Aadit, Thaddeus
